In [ ]:
import pandas as pd
import re
import numpy as np
import statsmodels.formula.api as smf
path_dist   = r"/Users/sanyakhan/oklahoma_gendered_mortality/data/input/abortion_distance_raw.csv"
path_births = r"/Users/sanyakhan/oklahoma_gendered_mortality/data/input/oklahoma_births-by-county_by-race_2010-2024.csv"

dist_raw   = pd.read_csv(path_dist)
births_raw = pd.read_csv(path_births)


In [2]:
births_raw.head(30)

,Search Characteristic,Values Selected,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Year 2010,NaN,NaN,NaN,NaN
1,NaN,NaN,Births,Population,Birth Rate
2,County of Residence,Mother's Race,NaN,NaN,NaN
3,UNKNOWN,White,18,.,.
4,NaN,Black or African American,.,.,.
5,NaN,American Indian or Alaska Native,.,.,.
6,NaN,Unknown,.,.,.
7,Adair,White,63,10038,6.3
8,NaN,Black or African American,.,69,.
9,NaN,American Indian or Alaska Native,169,10210,16.6


In [3]:
dist = dist_raw.copy()
# Option A: use the state column
ok = dist[dist["origin_state"] == "OK"].copy()
ok["county"] = (
    ok["origin_county_name"]
        .str.replace(r"\s*\(.*\)", "", regex=True)
        .str.replace(" County", "", regex=False)    
        .str.strip()
)
dist_df = ok[['year','month',"origin_fips_code", "county", "origin_state", "origin_population", 'distance_origintodest']]

dist_df

,year,month,origin_fips_code,county,origin_state,origin_population,distance_origintodest
426200,2010,8,40001,Adair,OK,4285.0,44.282017
426201,2010,9,40001,Adair,OK,4285.0,44.282017
426202,2010,10,40001,Adair,OK,4285.0,44.282017
426203,2010,11,40001,Adair,OK,4285.0,44.282017
426204,2010,12,40001,Adair,OK,4285.0,44.282017
...,...,...,...,...,...,...,...
441595,2018,1,40153,Woodward,OK,3669.0,141.880800
441596,2018,2,40153,Woodward,OK,3669.0,141.880800
441597,2018,3,40153,Woodward,OK,3669.0,141.880800
441598,2018,4,40153,Woodward,OK,3669.0,141.880800


In [4]:
import pandas as pd
import numpy as np

df = births_raw.copy()
df.columns = ["col0", "col1", "col2", "col3", "col4"]
is_year_row = df["col0"].astype(str).str.contains("Year", na=False)

df.loc[is_year_row, "year"] = (
    df.loc[is_year_row, "col0"]
      .astype(str)
      .str.extract(r"(\d{4})")[0]
      .astype("Int64")
)
df["year"] = df["year"].ffill()
df = df[df["year"].notna()].copy()
df = df[~df["col1"].eq("Mother's Race")].copy()
df = df[~is_year_row].copy()

df["county"] = df["col0"].where(df["col0"].notna()).ffill()

county_filter = (
    df["county"].fillna("").str.fullmatch("Unknown", case=False) |
    df["county"].fillna("").str.contains("UNKNOWN COUNTY", case=False)
)
df = df[~county_filter].copy()
df["race"] = df["col1"]

for c in ["col2", "col3", "col4"]:
    df[c] = pd.to_numeric(df[c].replace(".", np.nan), errors="coerce")

df = df.rename(columns={
    "col2": "births",
    "col3": "population",
    "col4": "birth_rate",
})
valid = df["births"].notna() | df["population"].notna()
df = df[valid].copy()

df.loc[df["births"].isna() & df["population"].notna(), "births"] = 0

needs_rate = df["birth_rate"].isna() & df["population"].gt(0)
df.loc[needs_rate, "birth_rate"] = (
    df.loc[needs_rate, "births"] / df.loc[needs_rate, "population"] * 1000
)
births_df = df[["year", "county", "race", "births", "population", "birth_rate"]].reset_index(drop=True)
births_df.head(40)


/var/folders/m_/y8pl7q8n0kgg5_f27n0x6vpm0000gn/T/ipykernel_63843/3958812945.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[~is_year_row].copy()


,year,county,race,births,population,birth_rate
0,2010,Adair,White,63.0,10038.0,6.3
1,2010,Adair,Black or African American,0.0,69.0,0.0
2,2010,Adair,American Indian or Alaska Native,169.0,10210.0,16.6
3,2010,Adair,Asian,0.0,136.0,0.0
4,2010,Adair,Unknown,131.0,NaN,NaN
5,2010,Adair,More than one race,5.0,2306.0,2.2
6,2010,Alfalfa,White,42.0,5101.0,8.2
7,2010,Alfalfa,American Indian or Alaska Native,0.0,174.0,0.0
8,2010,Alfalfa,More than one race,0.0,102.0,0.0
9,2010,Atoka,White,123.0,10596.0,11.6


In [5]:
births_df["county"] = births_df["county"].str.strip().str.lower()
dist_df["county"] = dist_df["county"].str.strip().str.lower()


/var/folders/m_/y8pl7q8n0kgg5_f27n0x6vpm0000gn/T/ipykernel_63843/168091081.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dist_df["county"] = dist_df["county"].str.strip().str.lower()


In [6]:

dist_year = (
    dist_df
    .groupby(["year", "county"], as_index=False)
    .agg({
        "distance_origintodest": "mean",  
        "origin_fips_code": "first",      
        "origin_state": "first",
        "origin_population": "first"      
    })
)

dist_year.rename(
    columns={
        "distance_origintodest": "dist_mean",
        "origin_population": "county_pop_dist"
    },
    inplace=True
)

births_merged = births_df.merge(
    dist_year[["year", "county", "origin_fips_code", "dist_mean"]],
    on=["year", "county"],
    how="left"
)

births_merged.head(20)


,year,county,race,births,population,birth_rate,origin_fips_code,dist_mean
0,2010,adair,White,63.0,10038.0,6.3,40001,41.352927
1,2010,adair,Black or African American,0.0,69.0,0.0,40001,41.352927
2,2010,adair,American Indian or Alaska Native,169.0,10210.0,16.6,40001,41.352927
3,2010,adair,Asian,0.0,136.0,0.0,40001,41.352927
4,2010,adair,Unknown,131.0,NaN,NaN,40001,41.352927
5,2010,adair,More than one race,5.0,2306.0,2.2,40001,41.352927
6,2010,alfalfa,White,42.0,5101.0,8.2,40003,117.173210
7,2010,alfalfa,American Indian or Alaska Native,0.0,174.0,0.0,40003,117.173210
8,2010,alfalfa,More than one race,0.0,102.0,0.0,40003,117.173210
9,2010,atoka,White,123.0,10596.0,11.6,40005,116.996120


In [7]:
county_year = (
    births_df
    .groupby(["year", "county"], as_index=False)
    .agg({
        "births": "sum",
        "population": "sum"
    })
)
county_year["birth_rate"] = (county_year["births"] / county_year["population"]) * 1000
county_year["county"] = county_year["county"].str.strip().str.lower()
dist_year["county"]    = dist_year["county"].str.strip().str.lower()
final_df = county_year.merge(
    dist_year[["year", "county", "dist_mean", "origin_fips_code"]],
    on=["year", "county"],
    how="left"
)

final_df["post_ban"] = (final_df["year"] >= 2022).astype(int)

final_df

,year,county,births,population,birth_rate,dist_mean,origin_fips_code,post_ban
0,2010,adair,368.0,22759.0,16.169427,41.352927,40001,0
1,2010,alfalfa,42.0,5377.0,7.811047,117.173210,40003,0
2,2010,atoka,167.0,14153.0,11.799618,116.996120,40005,0
3,2010,beaver,63.0,5414.0,11.636498,230.243500,40007,0
4,2010,beckham,326.0,22054.0,14.781899,122.553670,40009,0
...,...,...,...,...,...,...,...,...
1150,2024,wagoner,929.0,91257.0,10.180041,179.042720,40145,1
1151,2024,washington,580.0,54016.0,10.737559,130.514463,40147,1
1152,2024,washita,115.0,10758.0,10.689719,264.817230,40149,1
1153,2024,woods,59.0,8291.0,7.116150,138.861540,40151,1


In [8]:
import statsmodels.formula.api as smf

reg_df = final_df.dropna(subset=["birth_rate", "dist_mean"]).copy()
reg_df["post_ban"] = (reg_df["year"] >= 2022).astype("int64")
reg_df["year_cat"] = reg_df["year"].astype(str)
reg_df["dist_mean"] = reg_df["dist_mean"].astype("float64")
reg_df["birth_rate"] = reg_df["birth_rate"].astype("float64")


In [9]:
model_pp = smf.ols(
    "birth_rate ~ dist_mean * post_ban",
    data=reg_df
).fit(cov_type="HC3")

print(model_pp.summary())


                            OLS Regression Results                            
Dep. Variable:             birth_rate   R-squared:                       0.049
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     24.43
Date:                Tue, 02 Dec 2025   Prob (F-statistic):           2.51e-15
Time:                        12:47:34   Log-Likelihood:                -2373.8
No. Observations:                1155   AIC:                             4756.
Df Residuals:                    1151   BIC:                             4776.
Df Model:                           3                                         
Covariance Type:                  HC3                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             11.9790      0

In [10]:
import statsmodels.formula.api as smf

reg_df = final_df.dropna(subset=["birth_rate"]).copy()

reg_df["post_ban"] = (reg_df["year"] >= 2022).astype(int)

model_simple = smf.ols(
    formula="birth_rate ~ post_ban",
    data=reg_df
).fit(cov_type="HC3")

print(model_simple.summary())


                            OLS Regression Results                            
Dep. Variable:             birth_rate   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     59.76
Date:                Tue, 02 Dec 2025   Prob (F-statistic):           2.32e-14
Time:                        12:47:34   Log-Likelihood:                -2377.8
No. Observations:                1155   AIC:                             4760.
Df Residuals:                    1153   BIC:                             4770.
Df Model:                           1                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.0634      0.064    188.304      0.0

In [11]:
missing_in_dist = (
    births_df[["year", "county"]]
    .drop_duplicates()
    .merge(dist_year[["year", "county"]], on=["year", "county"], how="left", indicator=True)
)

missing_in_dist[missing_in_dist["_merge"] == "left_only"]


,year,county,_merge


In [12]:
formula = "birth_rate ~ dist_mean * post_ban + C(county_fips) + C(year)"

model_fe = smf.ols(
    formula=formula,
    data=reg_df
).fit(
    cov_type="cluster",
    cov_kwds={"groups": reg_df["county_fips"]}
)

print(model_fe.summary())

TypeError: Cannot interpret 'Int64Dtype()' as a data type